In [ ]:
from statLidl import readData 
df = readData()
print(df.head(5))

# Kunden insgesamt 
customers_total = df["Kundennr"].nunique()
print("Gesamtzahl Kunden: ", customers_total)
orders_total = df["Auftragsnummer"].nunique()
print("Gesamtzahl Bestellungen: ", orders_total)
mean_orders_per_customer = orders_total/customers_total
print("Durchschnittliche Anzahl Bestellungen pro Kunde.: ", mean_orders_per_customer)

from statLidl import grouping 
#merged_df = inspect(df, method="mean")
print("Grouped dataframe with summed values")
merged_df = grouping(df)

print(merged_df)
merged_df.to_excel('groupedDF.xlsx', sheet_name='Sheet1', index=False)


In [ ]:
"""
some general information about the dataframe of interest 
"""
print(df.columns)
print("Warengruppen: ", df["warengruppe"].unique(),  df["warengruppe"].nunique())
print("Anzahl Kunden: ", df["Kundennr"].nunique())
print("Wochen mit Jahr: ", df["weekyear"].unique(), df["weekyear"].nunique())
print("Wochen im jeweiligen Jahr: ")
print(df.groupby("year")["week"].nunique())
print("Kunden im jeweiligen Jahr: ")
print(df.groupby("year")["Kundennr"].nunique())
print("Bestellungen im Jahr: ")
print(df.groupby("year")["Auftragsnummer"].nunique())
print("Umsatz im jeweiligen Jahr: ")
print(df.groupby("year")["turnover"].sum())
print("Profit im jeweiligen Jahr: ")
print(df.groupby("year")["profit"].sum())
print("Durchschnittlicher Umsatzwert in der jeweiligen Woche: ")
average_turnover_per_week = df.groupby(['week'])['turnover'].mean()#.reset_index()
print(average_turnover_per_week)
print("Durchschnittlicher Umsatzwert pro Woche pro Bestellung: ")
average_turnover_per_week_per_sell = df.groupby(['week','Auftragsnummer'])['turnover'].mean()#.reset_index()
print(average_turnover_per_week_per_sell)

print("durchschnittliche anzahl bestellungen pro kunde pro woche")
average_orders_per_week_year = df.groupby(['yearweek', 'Kundennr'])['Auftragsnummer'].nunique().reset_index()
# Gruppiere nach Jahr und Woche, und berechne den Durchschnitt der Bestellanzahl pro Gruppe
average_orders_per_week_year = average_orders_per_week_year.groupby(['yearweek'])['Auftragsnummer'].mean().reset_index()
print(average_orders_per_week_year)

In [ ]:
def barPlot(df, col):
    import seaborn as sns 
    import matplotlib.pyplot as plt
    save = True
    title = "Coupon analysis"
    cat = 'yearweek'
    information_label = "using LIDL data"

    sns.set(style="whitegrid", font_scale=1.2, palette="viridis")
    plt.figure(figsize=(10, 6))
    xlabel = cat
    if cat == "yearweek":
        xlabel = "year & week"
        df[cat] = df[cat].astype('category')

    barplot = sns.barplot(x=cat, y=col, data=df, color=sns.color_palette()[0])
    max_value = df[col].max()
    min_value = df[col].min()

    ylabel = col.replace("_"," ")
    if col == "Auftragsnummer":
        ylabel = "number of orders" #"Anzahl Aufträge"
    elif col == "Kundennr":
        ylabel = "number of customers"# "Anzahl Kunden"
    elif col == "coupon":
        ylabel == "number of orders with coupon"

    # Customize plot elements
    plt.title(title, fontsize=12)
    plt.xlabel(xlabel, fontsize=14)
    plt.ylabel(ylabel, fontsize=14)
    plt.xticks(fontsize=12, rotation=45)
    plt.yticks(fontsize=12)
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.ylim(min_value - 0.1 * (max_value - min_value), max_value + 0.1 * (max_value - min_value))

    if 1:
        # Adding data labels
        for index, row in df.iterrows():
            if row[col]>100:
                label_rounded = f"{row[col]:.0f}"
            else:
                label_rounded = f"{row[col]:.1f}"
            plt.text(index, row[col], label_rounded, color='black', ha='center', alpha=0.6, fontsize=7)

    if 1:
        # Adding source information
        plt.text(
            0.98,
            -0.25,
            information_label,
            transform=plt.gca().transAxes,
            fontsize=10,
            color='gray',
            ha='right'
        )

    # Show the plot
    plt.tight_layout()
    if save:
        plt.savefig("barPlot" + "_" + col + ".png")
    else:
        plt.show()



bar_vars = ['turnover_sum', 'turnover_mean', 'profit_sum',
       'profit_mean', 'return_value_sum', 'return_value_mean',
       'cancellation_value_sum', 'cancellation_value_mean',
        'Auftragsnummer', 'Kundennr',
       'coupon', 'avg_orders_per_customer', 'avg_turnover_per_customer',
       'avg_turnover_per_order', 'avg_cancellation_value_per_customer',
       'avg_return_value_per_customer']
for var in bar_vars:
    print("barplot  for %s" % var)
    barPlot(df=merged_df, col=var)

In [ ]:

import matplotlib.pyplot as plt
import numpy as np
from scipy import stats
from scipy.stats import mannwhitneyu

def dfToDic(df, key_column, value_column):
    """
    transforms padndas dataframe df to dicionary given two columns for key and value
    """
    print(df.columns)
    dic = df.set_index(key_column)[value_column].to_dict()
    return dic
    
def Average(lst):
    """
    returns arithmetic mean of list lst
    """
    return sum(lst) / len(lst)

def getYesOrNo(bval):
    if bval == True or bval == "True":
        return "yes"
    elif bval ==  False or bval == "False":
        return "no"
    else:
        return ""

def runTTest(data_list, single_value, alpha = 0.05, name="saveaddition"):
    data = np.array(data_list)
    t_statistic, p_value = stats.ttest_1samp(data, single_value)
    mwu_stat, mwu_p = mannwhitneyu(data, single_value)


    print("\n\n\nrunning t-test for\n value: %s" % (single_value))
    print("data: ", data)
    sig_dev = False
    if p_value < alpha:
        print("The value deviates significantly from the set of values.")
        sig_dev = True
    else:
        print("The value does not deviate significantly from the set of values.")
        sig_dev = False

    print("name, t-statistic, p-value, alpha, sign. dev., MWU stat, MWU p")
    print(str(name) + ", " + str(t_statistic) + ", " + str(p_value) + ", " + str(alpha) + ", " + str(sig_dev) + ", " + str(mwu_stat) + ", " + str(mwu_p))


    plt.figure(figsize=(10, 6))
    plt.hist(data, bins=4, alpha=0.7, density=False, label="Data")
    plt.axvline(x=single_value, color='black', linestyle='-', linewidth=1.5, label="KW 42")
    plt.axvline(x=Average(data), color='gray', linestyle='dashed', linewidth=1.5, label="Mean KW!=42")
    if 1:
        plt.text(
            0.98,
            -0.15,
            "sign. deviation: %s" % getYesOrNo(sig_dev),
            transform=plt.gca().transAxes,
            fontsize=10,
            color='gray',
            ha='right'
        )
        plt.text(
            0.98,
            -0.2,
            "alpha: %.2f" % alpha,
            transform=plt.gca().transAxes,
            fontsize=10,
            color='gray',
            ha='right'
        )
        plt.text(
            0.98,
            -0.25,
            "p-value: %.4f" % p_value,
            transform=plt.gca().transAxes,
            fontsize=10,
            color='gray',
            ha='right'
        )
        plt.text(
            1.14,
            -0.25,
            "using LIDL data",
            transform=plt.gca().transAxes,
            fontsize=10,
            color='gray',
            ha='right'
        )
    xlabel = name.split("-")[0].replace("_"," ").replace("avg","average")
    plt.xlabel(xlabel, fontsize=14)
    plt.ylabel("Number of entries per bin", fontsize=14)
    title = name.replace("_"," ")
    if "KW42nc" in title: 
        title = "KW42 2020 no coupon vs. other weeks"
    elif "KW42c" in title: 
        title = "KW42 2020 with coupon vs. other weeks"
    elif "KW42" in title: 
        title = "KW42 2020 vs. other weeks"
    title +=  " (without KW42 2019)"""
    plt.title("t-test" + " " + title,  fontsize=12)
    legend = plt.legend(fontsize=10,loc="best",bbox_to_anchor=(1, 1))
    legend.get_frame().set_alpha(0)  
    plt.xticks(fontsize=12, rotation=45)
    plt.yticks(fontsize=12)
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.grid(axis='x', linestyle='--', alpha=0.0)
    plt.tight_layout()
    plt.savefig("ttest" + name + ".png")

def statTTest(df, var):
    dict = dfToDic(df, "yearweek", var)
    all = dict["2020-42"]
    nc = dict["2020-42-nc"]
    c = dict["2020-42-c"]
    del dict["2020-42-nc"]
    del dict["2020-42"]
    del dict["2020-42-c"]

    # also delete 19 values
    try:
        print("deleting 2019 KW 42 data")
        del dict["2019-42-nc"]
        del dict["2019-42"]
        del dict["2019-42-c"]
    except:
        print("deletion of 2019 data in KW 42 not possible")

    values = list(dict.values())
    alpha = 0.05
    runTTest(values, all, alpha, "%s-KW42" % var)
    runTTest(values, nc, alpha, "%s-KW42nc" % var)
    runTTest(values, c, alpha, "%s-KW42c" % var)

vars_to_test = ["turnover_sum", "turnover_mean", "cancellation_value_sum", "cancellation_value_mean", "return_value_sum", "return_value_mean", "profit_sum", "profit_mean", "avg_orders_per_customer","avg_turnover_per_customer","avg_turnover_per_order","avg_cancellation_value_per_customer","avg_return_value_per_customer","Auftragsnummer","Kundennr"]
for var in vars_to_test:
    statTTest(merged_df, var)

In [ ]:

# Kunden, die in der jeweiligen Woche neu hinzugekommen sind
import pandas as pd

def newCustomersPerWeek2(df):
        df = df.sort_values(by=['Kundennr', 'week'])
        unique_customers_per_week = df.groupby('week')['Kundennr'].apply(set)
        new_customers_per_week = []
        all_customers = set()
        for week, customers in unique_customers_per_week.items():
            new_customers = customers - all_customers
            new_customers_per_week.append(len(new_customers))
            all_customers.update(customers)
        all_weeks = range(min(df['week']), max(df['week']) + 1)
        result = pd.DataFrame({'week': all_weeks, 'new_customers': new_customers_per_week})
        print("Anzahl neu hinzugekommener Kunden pro Woche:\n", result)
        return result

new_customers_per_week = newCustomersPerWeek2(df)

print("Did customers bought more than twice? How often?\n")
customer_purchase_counts = df['Kundennr'].value_counts()
print(customer_purchase_counts.head(10))

def firstWeekOfPurchase(df):
        """
        get first week of purchase of customers
        """
        kundendaten = df
        dategroup = "week"
        first_week_of_purchase = kundendaten.groupby('Kundennr')[dategroup].min()
        print(first_week_of_purchase)
        print("First week of purchase for customers with multiple purchases:\n", first_week_of_purchase)
        plt.figure(figsize=(10, 6))
        first_week_of_purchase.value_counts().sort_index().plot(kind='bar')
        plt.xlabel('%s' % dategroup, fontsize=14)
        plt.ylabel('number of customers', fontsize=14)
        plt.title('Number of customers with first purchase per week', fontsize=12)
        plt.xticks(fontsize=12, rotation=45)
        plt.yticks(fontsize=12)
        plt.text(
            0.98,
            -0.25,
            "using LIDL data",
            transform=plt.gca().transAxes,
            fontsize=10,
            color='gray',
            ha='right'
        )
        plt.tight_layout()
        plt.savefig("numberOfCustomersFirstPurchasePerWeek.png")
        return first_week_of_purchase

firstWeekOfPurchase(df)

def firstWeekOfPurchaseMorethanTwice(df):
        """
        get first week of purchase of customers who bought more than one time (at least two times)
        """
        kundendaten = df
        dategroup = "week"
        purchase_counts = kundendaten.groupby('Kundennr')[dategroup].count()

        # Filter customers who bought at least twice
        multiple_purchase_customers = purchase_counts[purchase_counts >= 2]

        # Get the first week of purchase for each customer who bought at least twice
        first_week_of_purchase = kundendaten[kundendaten['Kundennr'].isin(multiple_purchase_customers.index)]
        first_week_of_purchase = first_week_of_purchase.groupby('Kundennr')[dategroup].min()

        print("First week of purchase for customers who bought more than once:\n", first_week_of_purchase)

        # Plot the data in a bar plot
        plt.figure(figsize=(10, 6))
        first_week_of_purchase.value_counts().sort_index().plot(kind='bar')
        plt.xlabel('%s' % dategroup, fontsize=14)
        plt.ylabel('number of customers', fontsize=14)
        plt.title('Number of customers with first purchase per week (orderd more than once)', fontsize=12)
        plt.xticks(fontsize=12, rotation=45)
        plt.yticks(fontsize=12)
        plt.grid(axis='y', linestyle='--', alpha=0.7)        
        plt.text(
            0.98,
            -0.25,
            "using LIDL data",
            transform=plt.gca().transAxes,
            fontsize=10,
            color='gray',
            ha='right'
        )
        plt.tight_layout()
        plt.savefig("numberOfCustomersFirstPurchasePerWeekMoreThanOnce.png")

        return first_week_of_purchase

firstWeekOfPurchaseMorethanTwice(df)

In [ ]:

pie_chart_warengruppe = 1
import seaborn as sns
if pie_chart_warengruppe:
    c = (99.64243146603098, 0.23837902264600713, 0.119)
    nc =  (99.62168978562421, 0.27742749054224464, 0.100)

    import matplotlib.pyplot as plt

    labels = ["W1", "W2", "W3"]

    # Calculate the sum of values for each tuple
    sum_c = sum(c)
    sum_nc = sum(nc)

    # Calculate the proportional values for the pie chart
    proportional_c = [value / sum_c * 100 for value in c]
    proportional_nc = [value / sum_nc * 100 for value in nc]

    # Create the pie chart
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))
    colors = sns.color_palette("Set3", n_colors=3)

    explode = (0.1, 0.0, 0.2)  # Pull the first slice (w1) outward
    sns.set(style="whitegrid", font_scale=1.2, palette="viridis")

    ax1.pie(proportional_c, labels=labels, autopct='%1.3f%%', startangle=45, explode=explode, shadow=True, colors=colors)
    ax1.set_title('coupon')

    ax2.pie(proportional_nc, labels=labels, autopct='%1.3f%%', startangle=45, explode=explode, shadow=True,colors=colors)
    ax2.set_title('no coupon')

    plt.title("Einfluss des Gutscheins auf Warengruppen in KW 42 in 2020")
    plt.tight_layout()
    #plt.show()
    plt.savefig("piechart_warengruppen" + ".png")


